In [ ]:
# Uncomment this if you are using Colab
# ## Mount google drive: If your dataset is saved on google drive
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt


# Step 1: Load the images

# b) Load the first image
imFirst = np.array(Image.open('./faces/s1/1.pgm'))
imFirst = imFirst.astype('float32')
imFirst /= 255.0
imFirst = np.clip(imFirst, 0.0, 1.0)
plt.imshow(imFirst, cmap='gray')
plt.show()

# c) Load the training images (40 persons X 7 images)
nTrainImages = 40 * 7  # number of training images
height, width = imFirst.shape  # The images have the same size. Use the first one to calculate the number of pixels
imagesTrain = np.zeros((height * width, nTrainImages))  # Matrix with the training images as rows
for i in range(1, 41):
    for j in range(1, 8):
        filename = f'./faces/s{i}/{j}.pgm'
        img = np.array(Image.open(filename))
        img = img.astype('float32')
        img /= 255.0
        img = np.clip(img, 0.0, 1.0)
        imagesTrain[:, (i-1)*7 + (j-1)] = img.flatten()

# d) read testing images.
nTestImages = 40 * 3  # testing images
imagesTest = np.zeros((height * width, nTestImages))
for i in range(1, 41):
    for j in range(8, 11):
        filename = f'./faces/s{i}/{j}.pgm'
        img = np.array(Image.open(filename))
        img = img.astype('float32')
        img /= 255.0
        img = np.clip(img, 0.0, 1.0)
        imagesTest[:, (i-1)*3 + (j-8)] = img.flatten()

#### Reference: Explaination of why use $A^T A$ for eigenvalue (answered by GPT4)

The relation between $A^T A$ and the covariance matrix of $A$, where $A$ is a matrix whose columns are random variables that follow a zero-mean distribution, is quite direct.

The covariance matrix $ \Sigma $ of $ A $ is defined as $ \Sigma = E[(A - \mu)(A - \mu)^T] $, where $ \mu $ is the mean vector of the columns of $ A $. Since in your case each column of $ A $ follows a zero-mean distribution, $ \mu = 0 $. Therefore, the covariance matrix simplifies to $ \Sigma = E[AA^T] $.

However, $ A^T A $ is a related but slightly different quantity. It is not the covariance matrix of the columns of $ A $, but rather a matrix that represents the dot products between all pairs of columns in $ A $. In the context of zero-mean data, $ A^T A $ can be seen as a scaled version of the covariance matrix when the data is centered but not necessarily scaled to unit variance. Specifically, if $ A $ is a data matrix with zero-mean columns, then $ A^T A $ divided by $ n-1 $, where $ n $ is the number of observations (rows in $ A $), gives the sample covariance matrix of the data in $ A $.

So, in summary, for zero-mean data:
- The covariance matrix $ \Sigma $ of $ A $ is $ E[AA^T] $.
- $ A^T A $ represents the sum of squares and cross-products matrix for the columns of $ A $.
- $ \frac{1}{n-1} A^T A $ gives the sample covariance matrix of $ A $ when $ A $ has zero-mean columns.

**But:** if you really encounter OOM issue for large dataset PCA, please refer to incremental PCA or other way to estimate the components.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Step 2: PCA
# a) find the mean image
mean_face = np.mean(imagesTrain, axis=1)

image_height, image_width = height, width
# TODO: visualize mean_face
plt.imshow(mean_face.reshape(image_height, image_width), cmap='gray')
plt.axis('off')
plt.show()

# b) mean-shifted input images
shifted_images = imagesTrain - mean_face[:, np.newaxis]

# This is a pre-step to optimize calculation. (see your lecture notes)
# TODO: normally we should find the covariance of shifted_images but it will be a 10304*10304 matrix!
# to be optimized and avoid "out of memory" error, we use a trick! (see chapter 6 of the book (p. 164)
# Compute Y'*Y 
YY = 1/shifted_images.shape[0] * np.dot(shifted_images.T, shifted_images)

# c) Compute eigenvectors
evalues, evectors = np.linalg.eig(YY)
evectors = np.dot(shifted_images, evectors)

# d) Sort eigenvectors based on their corresponding eigenvalues
# TODO: put the results in "evectors" variable
sorted_indices = np.argsort(evalues)[::-1]
evectors = evectors[:, sorted_indices]
evalues = evalues[sorted_indices]

# e) only retain the top 'num_eigenfaces' eigenvectors (i.e. the principal components)
num_eigenfaces = 30
evectors = evectors[:, :num_eigenfaces]

# Normalize the eigenvectors so || evector_i|| = 1
for i in range(num_eigenfaces):
    evectors[:, i] /= np.linalg.norm(evectors[:, i])

# f) project the images into the subspace to generate the feature vectors
# TODO: put the results in "features" variable
features = np.dot(shifted_images.T, evectors)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Step 3: Analysis of eigen faces
# a) display the eigenvectors
fig, axes = plt.subplots(1, num_eigenfaces, figsize=(12, 4))
for n in range(num_eigenfaces):
    evector = evectors[:, n].reshape(height, width)
    axes[n].imshow(evector, cmap='gray')
    axes[n].axis('off')
plt.show()

In [ ]:
# b) display the cumulative eigenvalues
cumulative_sum = np.cumsum(evalues)
plt.plot(cumulative_sum / np.sum(evalues))
plt.xlabel('Number of eigenfaces')
plt.ylabel('Cumulative sum of eigenvalues')
plt.show()

In [ ]:
# Step 4: Identify a person
testIm = 100 # chose a person index
input_image = np.expand_dims(imagesTest[:, testIm],-1)
# Tips: a shortcut way in python is input_image = imagesTest[:, testIm][..., None]

shifted_input_image = input_image - mean_face[:, np.newaxis]
shifted_imagesTrain = imagesTrain - mean_face[:, np.newaxis]

# calculate the similarity of the input to each training image
# feature_vec = np.dot(input_image.T, evectors)
input_image_feature =  np.dot(shifted_input_image.T, evectors)
imagesTrain_feature =  np.dot(shifted_imagesTrain.T, evectors)

similarity_score = np.zeros((len(imagesTrain_feature)))
for i in range(0,len(imagesTrain_feature)):
    similarity_score[i] = 1/(1+np.linalg.norm(input_image_feature-imagesTrain_feature[i])) # norm can be regarded as distance between two vectors

# find the image with the highest similarity
match_ix = np.argmax(similarity_score)

# display the result
fig, ax = plt.subplots(1, 2)
ax[0].imshow(input_image.reshape(height, width), cmap='gray')
ax[0].axis('off')
ax[1].imshow(imagesTrain[:, match_ix].reshape(height, width), cmap='gray')
ax[1].axis('off')
ax[1].set_title(f"matches to the {match_ix}th training sample with score {similarity_score[match_ix]}")
plt.show()

In [ ]:
# Step 5: Cluster the persons

# a) Cluster the original images
# put the results in "clusters" variable. "clusters" indicates cluster number for each sample.
# hint: use kmeans function
from sklearn.cluster import KMeans
NUM_PERSONS = 40
kmeans = KMeans(n_clusters=NUM_PERSONS)
kmeans.fit(imagesTrain.T)
clusters = kmeans.labels_

# Show the the images that were assigned to cluster 1
img_index = np.where(clusters == 0)[0]

for i in range(min(len(img_index), 7)):
    plt.figure(111)
    plt.subplot(1, 7, i+1)
    img = np.reshape(imagesTrain[:, img_index[i]], (height, width))
    plt.imshow(img, cmap='gray')
    plt.axis('off')

In [ ]:
# b) Now cluster on the reduced space
# put the results in "clusters_pca" variable.
kmeans_pca = KMeans(n_clusters=NUM_PERSONS)
kmeans_pca.fit(features)
clusters_pca = kmeans_pca.labels_

# Show the the images that were assigned to cluster 1
img_index = np.where(clusters_pca == 0)[0]
for i in range(min(len(img_index), 7)):
    plt.figure(111)
    plt.subplot(1, 7, i+1)
    img = np.reshape(imagesTrain[:, img_index[i]], (height, width))
    plt.imshow(img, cmap='gray')
    plt.axis('off')

In [ ]:
# c) Try agglomerative clustering on some images
# hint: use linkage command for agglomerative clustering with "single" metric and put the results in "Z_agglo" variable
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

checkImages = list(range(8,21))

Z_agglo = linkage(imagesTrain.T[checkImages], method='single')

fig = plt.figure()
dn = dendrogram(Z_agglo)
c = fcluster(Z_agglo, t=2, criterion='maxclust')

In [ ]:
# Show the the images that were assigned to cluster 1
img_index = np.where(c == 1)[0]
for i in range(min(len(img_index), 5)):
    plt.figure(111)
    plt.subplot(1, 5, i+1)
    img = np.reshape(imagesTrain[:, checkImages[img_index[i]]], (height, width))
    plt.imshow(img, cmap='gray')
    plt.axis('off')

### QA

1. how to implement python-style image batching

In [ ]:
import numpy as np

arr = []
for i in range(5):
    a = np.zeros((10,20,3)) # for instance a is the loaded image
    print(f"image size at stage {i} is {a.shape}")
    arr.append(a)
arr_np = np.stack(arr)
print("batched arr size", arr_np.shape)
print("flatten images in the batch", arr_np.reshape(arr_np.shape[0], -1).shape)
